In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# source : http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
# Small LSTM Network to Generate Text 
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase

print ('Loading file into raw text ...')
filename = "cleaned_dataset_dataset.txt"
raw_text = open(filename).read()

print ('Turning text into lowercase ...')
raw_text = raw_text.lower()

# create mapping of unique chars to integers
print ('Create mapping chars into integer ...')
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
print ('Prepare the dataset of input to output pairs encoded as integers ...')
seq_length = 50
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
print ('reshape X to be [samples, time steps, features]')
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
print ('Normalize ...')
X = X / float(n_vocab)


# one hot encode the output variable
print ('one hot encode the output variable')
y = np_utils.to_categorical(dataY)


# define the LSTM model
print ('define the LSTM model')
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

print ('Compile model')
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
print ('define the checkpoint')
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
print('Fit model')
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

/home/ncarkaci/anaconda3/lib/python3.6/site-packages/numpy/core/machar.py:127: RuntimeWarning: overflow encountered in add
  a = a + a
/home/ncarkaci/anaconda3/lib/python3.6/site-packages/numpy/core/machar.py:129: RuntimeWarning: invalid value encountered in subtract
  temp1 = temp - a
/home/ncarkaci/anaconda3/lib/python3.6/site-packages/numpy/core/machar.py:138: RuntimeWarning: invalid value encountered in subtract
  itemp = int_conv(temp-a)
/home/ncarkaci/anaconda3/lib/python3.6/site-packages/numpy/core/machar.py:162: RuntimeWarning: overflow encountered in add
  a = a + a
/home/ncarkaci/anaconda3/lib/python3.6/site-packages/numpy/core/machar.py:164: RuntimeWarning: invalid value encountered in subtract
  temp1 = temp - a
/home/ncarkaci/anaconda3/lib/python3.6/site-packages/numpy/core/machar.py:171: RuntimeWarning: invalid value encountered in subtract
  if any(temp-a != zero):
Using TensorFlow backend.


Loading file into raw text ...
Turning text into lowercase ...
Create mapping chars into integer ...
Total Characters:  3820313
Total Vocab:  40
Prepare the dataset of input to output pairs encoded as integers ...
Total Patterns:  3820263
reshape X to be [samples, time steps, features]
Normalize ...
one hot encode the output variable
define the LSTM model
Compile model
define the checkpoint
Fit model
Epoch 1/50
   9280/3820263 [..............................] - ETA: 13461s - loss: 3.1456

KeyboardInterrupt: 